# **Electric Vehicle Population Data Cleaning**

In [72]:
import pandas as pd
import sqlite3


In [73]:
ev_pop_df = pd.read_csv(r'Electric_Vehicle_Population_Data.csv')
ev_pop_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         177866 non-null  object 
 1   County                                             177861 non-null  object 
 2   City                                               177861 non-null  object 
 3   State                                              177866 non-null  object 
 4   Postal Code                                        177861 non-null  float64
 5   Model Year                                         177866 non-null  int64  
 6   Make                                               177866 non-null  object 
 7   Model                                              177866 non-null  object 
 8   Electric Vehicle Type                              177866 non-null  object

### **Change the column names**

In [74]:
ev_pop_df.columns = (ev_pop_df.columns
                     .str.replace(' ','_')
                     .str.lower()
                     )
ev_pop_df = (ev_pop_df
             .rename(columns={'vin_(1-10)':'vin',
                              'electric_vehicle_type':'ev_type',
                              'clean_alternative_fuel_vehicle_(cafv)_eligibility':'cafv_eligibility',
                              'electric_range':'range',
                              'legislative_district':'district',
                              'vehicle_location':'location'
                              }
                     )
             )
ev_pop_df.info()  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   vin                177866 non-null  object 
 1   county             177861 non-null  object 
 2   city               177861 non-null  object 
 3   state              177866 non-null  object 
 4   postal_code        177861 non-null  float64
 5   model_year         177866 non-null  int64  
 6   make               177866 non-null  object 
 7   model              177866 non-null  object 
 8   ev_type            177866 non-null  object 
 9   cafv_eligibility   177866 non-null  object 
 10  range              177866 non-null  int64  
 11  base_msrp          177866 non-null  int64  
 12  district           177477 non-null  float64
 13  dol_vehicle_id     177866 non-null  int64  
 14  location           177857 non-null  object 
 15  electric_utility   177861 non-null  object 
 16  20

### **Preview the data**

In [75]:
ev_pop_df.head()


,vin,county,city,state,postal_code,model_year,make,model,ev_type,cafv_eligibility,range,base_msrp,district,dol_vehicle_id,location,electric_utility,2020_census_tract
0,5YJYGDEE1L,King,Seattle,WA,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,POINT (-122.30839 47.610365),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,7SAYGDEE9P,Snohomish,Bothell,WA,98021.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1.0,244285107,POINT (-122.179458 47.802589),PUGET SOUND ENERGY INC,5.306105e+10
2,5YJSA1E4XK,King,Seattle,WA,98109.0,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,270,0,36.0,156773144,POINT (-122.34848 47.632405),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,5YJSA1E27G,King,Issaquah,WA,98027.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,5.0,165103011,POINT (-122.03646 47.534065),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,5YJYGDEE5M,Kitsap,Suquamish,WA,98392.0,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,23.0,205138552,POINT (-122.55717 47.733415),PUGET SOUND ENERGY INC,5.303594e+10


### **A focus on `electric_utility` column**

In [76]:
ev_pop_df.electric_utility.unique()


array(['CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA)',
       'PUGET SOUND ENERGY INC',
       'PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)', 'PACIFICORP',
       'PUD NO 1 OF CHELAN COUNTY', 'MODERN ELECTRIC WATER COMPANY',
       'PUD NO 2 OF GRANT COUNTY', 'AVISTA CORP',
       'PUD NO 1 OF PEND OREILLE COUNTY',
       'BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF CLARK COUNTY - (WA)',
       'BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF COWLITZ COUNTY',
       'BONNEVILLE POWER ADMINISTRATION||PUD 1 OF SNOHOMISH COUNTY',
       'BONNEVILLE POWER ADMINISTRATION||PUGET SOUND ENERGY INC||PUD NO 1 OF JEFFERSON COUNTY',
       'NO KNOWN ELECTRIC UTILITY SERVICE',
       'BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF MASON COUNTY|PUD NO 1 OF JEFFERSON COUNTY',
       'BONNEVILLE POWER ADMINISTRATION||PACIFICORP||PUD NO 1 OF CLARK COUNTY - (WA)',
       'BONNEVILLE POWER ADMINISTRATION||PACIFICORP||BENTON RURAL ELECTRIC ASSN',
       'BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF CLALLAM 

> Cleaning the `electric_utility` by removing `|` from the entries and add more columns for vehicles that had more than one charging station

In [77]:
def clean_electric_utility(ev_pop_df):
    electric_utility_col = ev_pop_df['electric_utility']
    electric_utility_col_split = (electric_utility_col
                                  .str.split(pat='|', expand=True)
                                  .add_prefix('electric_utility_')
                                  )
    ev_pop_df = (pd
                 .concat([ev_pop_df,electric_utility_col_split],axis=1)
                 .drop(columns='electric_utility')
                 )
    return ev_pop_df
ev_pop_df = clean_electric_utility(ev_pop_df.copy())
ev_pop_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   vin                 177866 non-null  object 
 1   county              177861 non-null  object 
 2   city                177861 non-null  object 
 3   state               177866 non-null  object 
 4   postal_code         177861 non-null  float64
 5   model_year          177866 non-null  int64  
 6   make                177866 non-null  object 
 7   model               177866 non-null  object 
 8   ev_type             177866 non-null  object 
 9   cafv_eligibility    177866 non-null  object 
 10  range               177866 non-null  int64  
 11  base_msrp           177866 non-null  int64  
 12  district            177477 non-null  float64
 13  dol_vehicle_id      177866 non-null  int64  
 14  location            177857 non-null  object 
 15  2020_census_tract   177861 non-nul

### **A focus on `location` column**

In [78]:
ev_pop_df['location'] = ev_pop_df['location'].str.strip('POINT()')
ev_pop_df[['longitude','latitude']] = ev_pop_df['location'].str.split(expand=True)
ev_pop_df['longitude'] = ev_pop_df['longitude'].str.strip('(').astype(float)
ev_pop_df['latitude'] = ev_pop_df['latitude'].astype(float)
ev_pop_df = ev_pop_df.drop(columns='location')
ev_pop_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   vin                 177866 non-null  object 
 1   county              177861 non-null  object 
 2   city                177861 non-null  object 
 3   state               177866 non-null  object 
 4   postal_code         177861 non-null  float64
 5   model_year          177866 non-null  int64  
 6   make                177866 non-null  object 
 7   model               177866 non-null  object 
 8   ev_type             177866 non-null  object 
 9   cafv_eligibility    177866 non-null  object 
 10  range               177866 non-null  int64  
 11  base_msrp           177866 non-null  int64  
 12  district            177477 non-null  float64
 13  dol_vehicle_id      177866 non-null  int64  
 14  2020_census_tract   177861 non-null  float64
 15  electric_utility_0  177861 non-nul

### **Partitioning Data to various table**

In [79]:
## Table 1
vehicles = ev_pop_df[['vin','make','model','model_year','ev_type','cafv_eligibility','range']]
vehicles = (vehicles
            .assign(model_year = lambda x: x.model_year.astype('int32'),
                    range = lambda x: x.range.astype('int32')
                    )
            )
vehicles.to_csv('vehicles.csv',index=False)
vehicles.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   vin               177866 non-null  object
 1   make              177866 non-null  object
 2   model             177866 non-null  object
 3   model_year        177866 non-null  int32 
 4   ev_type           177866 non-null  object
 5   cafv_eligibility  177866 non-null  object
 6   range             177866 non-null  int32 
dtypes: int32(2), object(5)
memory usage: 8.1+ MB


In [80]:
## Table 2
location = ev_pop_df[['vin','longitude','latitude','county','city']]
location = (location
            .assign(longitude=lambda x: x.longitude.astype('float64'),
                    latitude=lambda x: x.latitude.astype('float64'),
                    )
            )
location.to_csv('location.csv',index=False)
location.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   vin        177866 non-null  object 
 1   longitude  177857 non-null  float64
 2   latitude   177857 non-null  float64
 3   county     177861 non-null  object 
 4   city       177861 non-null  object 
dtypes: float64(2), object(3)
memory usage: 6.8+ MB


In [81]:
## Table 3
charging_stations = ev_pop_df[['vin','electric_utility_0','electric_utility_1',
                               'electric_utility_2','electric_utility_3',
                               'electric_utility_4','electric_utility_5']]
charging_stations.to_csv('charging_stations.csv', index=False)
charging_stations.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   vin                 177866 non-null  object
 1   electric_utility_0  177861 non-null  object
 2   electric_utility_1  136129 non-null  object
 3   electric_utility_2  104748 non-null  object
 4   electric_utility_3  14674 non-null   object
 5   electric_utility_4  14317 non-null   object
 6   electric_utility_5  1117 non-null    object
dtypes: object(7)
memory usage: 9.5+ MB


### **Store data in a sql database**

In [82]:
vehicles = pd.read_csv('vehicles.csv')
location = pd.read_csv('location.csv')
charging_stations = pd.read_csv('charging_stations.csv')


In [83]:
conn = sqlite3.connect('washington_electric_vehicle_population.db')
vehicles.to_sql('vehicles', conn,if_exists='replace',index=False)
location.to_sql('location',conn,if_exists='replace',index=False)
charging_stations.to_sql('charging_stations',conn,if_exists='replace',index=False)
conn.close()


## **Cleaning the history dataset to only accommodate 2020 data**

In [84]:
history_data = pd.read_csv(r'Electric_Vehicle_Population_Size_History_20240418.csv')
history_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Date                                          86 non-null     object
 1   Plug-In Hybrid Electric Vehicle (PHEV) Count  86 non-null     int64 
 2   Battery Electric Vehicle (BEV) Count          86 non-null     int64 
 3   Electric Vehicle (EV) Total                   86 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 2.8+ KB


> A focus on the `Date` column to ensure the correct data formatting is being used


In [85]:
history_data['Date'] = pd.to_datetime(history_data['Date'], format='%B %d %Y')
history_data = (history_data
                .assign(Date = lambda x: x.Date.dt.date)
                )
history_data['Date'] = pd.to_datetime(history_data['Date'])
history_data.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Date                                          86 non-null     datetime64[ns]
 1   Plug-In Hybrid Electric Vehicle (PHEV) Count  86 non-null     int64         
 2   Battery Electric Vehicle (BEV) Count          86 non-null     int64         
 3   Electric Vehicle (EV) Total                   86 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 2.8 KB


In [86]:
history_data = (history_data.rename(columns={
    'Date':'date',
    'Plug-In Hybrid Electric Vehicle (PHEV) Count':'phev_count',
    'Battery Electric Vehicle (BEV) Count':'bev_count',
    'Electric Vehicle (EV) Total':'ev_total_count',
                                             }
                                    )
                )
history_data.to_csv('history_data.csv',index=False)
history_data.head()


,date,phev_count,bev_count,ev_total_count
0,2024-02-29,38656,139210,177866
1,2024-01-31,38166,136494,174660
2,2023-12-31,37142,132362,169504
3,2023-11-30,36533,129084,165617
4,2023-10-31,35888,125977,161865
